Exploração das fontes de dados abertos a partir do [código de Análise de Redes](https://colab.research.google.com/drive/1P1cRwARdDxXYy_JMTmEKs0pT4SR4DQNT?usp=sharing).

<H1> Importar pacotes

In [1]:
import requests
import json
import pandas as pd

<H1> Coleta da Pesquisa Básica das Matérias
<H3> (PLs do Senado de 2019)

In [2]:
PARAMS = {
    "sigla":"pdl", #pl
    "ano":"2019"
        }

HEADERS = {
    "Accept":"application/json"
}
URL = "https://legis.senado.leg.br/dadosabertos/materia/pesquisa/lista"
response = requests.get(url=URL,params=PARAMS,headers=HEADERS).json()

In [3]:
## conferir chaves
for key in response.keys():
    print(key)

PesquisaBasicaMateria


In [4]:
list_pdl = list(response["PesquisaBasicaMateria"]["Materias"]["Materia"])
#list_pls = list(response["PesquisaBasicaMateria"]["Materias"]["Materia"])

In [5]:
df_pdl = pd.DataFrame(list_pdl)
#df_pls = pd.DataFrame(list_pls)

In [6]:
df_pdl.head()

,Codigo,IdentificacaoProcesso,DescricaoIdentificacao,Sigla,Numero,Ano,Ementa,Autor,Data,UrlDetalheMateria
0,135086,7709581,PDL 1/2019,PDL,00001,2019,Aprova o ato que outorga autorização à Associa...,Câmara dos Deputados,2019-01-30,http://legis.senado.leg.br/dadosabertos/materi...
1,135107,7709135,PDL 16/2019,PDL,00016,2019,"Susta os efeitos do Decreto 9.661, editado pel...","Senador Paulo Rocha (PT/PA), Senador Jean Paul...",2019-01-04,http://legis.senado.leg.br/dadosabertos/materi...
2,135113,7711518,PDL 17/2019,PDL,00017,2019,"Altera o Decreto Legislativo nº 276, de 2014, ...",Senador Flávio Arns (REDE/PR),2019-02-07,http://legis.senado.leg.br/dadosabertos/materi...
3,135153,7716780,PDL 23/2019,PDL,00023,2019,"Susta o Decreto nº 9.685, de 15 de janeiro de ...","Senador Humberto Costa (PT/PE), Senador Jean P...",2019-02-12,http://legis.senado.leg.br/dadosabertos/materi...
4,135154,7716759,PDL 22/2019,PDL,00022,2019,"Susta o Decreto nº 9.690, de 23 de janeiro de ...","Senador Humberto Costa (PT/PE), Senador Jean P...",2019-02-12,http://legis.senado.leg.br/dadosabertos/materi...


**decisão**: <br>
projetos de decreto (PDL ou PDS) nao serão incluidos na análise, por: 1) não criam novas leis, apenas regulamentam as existentes;   2) apresentam menor "grau de mudança" visto que não precisam de sanção do presidente nem seguem regras de votações mais rigorosas como as PECs.

In [ ]:
#checar se tem projeto de comissão (iniciativa coletiva)
df_pl_2019 = pd.DataFrame(list(response["PesquisaBasicaMateria"]["Materias"]["Materia"]))
df_pl_2019_comissao = df_pl_2019[df_pl_2019['Autor'].str.contains("Comiss")]
df_pl_2019_comissao

In [ ]:
#checar projetos do Senado
df_pl_2019_sen = df_pl_2019[df_pl_2019['Autor'].str.contains("Senador")]
df_pl_2019_sen

In [ ]:
#checar projetos da CD
df_pl_2019_cd = df_pl_2019[df_pl_2019['Autor'].str.contains("Câmara dos Deputados")]
df_pl_2019_cd

In [ ]:
#checar autores diferentes da Câmara dos Deputados
df_pl_2019_notcd = df_pl_2019[~df_pl_2019['Autor'].str.contains("Câmara dos Deputados")]
df_pl_2019_notcd

In [ ]:
print("projetos de comissao:", len(df_pl_2019_comissao.index))
print("projetos de senadores individuais:", len(df_pl_2019_sen.index))
print("projetos da Câmara dos Dep.:", len(df_pl_2019_cd))
print("projetos que nao da Câmara dos Dep.:", len(df_pl_2019_notcd.index))
print("projetos totais de 2019:", len(df_pl_2019.index))

projetos de comissao: 13
projetos de senadores individuais: 1149
projetos da Câmara dos Dep.: 336
projetos que nao da Câmara dos Dep.: 1165
projetos totais de 2019: 1501


In [ ]:
df_pl_2019_sen_comissao.columns

Index(['Codigo', 'IdentificacaoProcesso', 'DescricaoIdentificacao', 'Sigla',
       'Numero', 'Ano', 'Ementa', 'Autor', 'Data', 'UrlDetalheMateria'],
      dtype='object')

In [ ]:
df_pl_2019_sen_comissao = pd.concat([df_pl_2019_sen, df_pl_2019_comissao])
df_all = df_pl_2019_notcd.merge(df_pl_2019_sen_comissao,
                                on=['Codigo', 'IdentificacaoProcesso', 'DescricaoIdentificacao', 'Sigla',
                                    'Numero', 'Ano', 'Ementa', 'Autor', 'Data', 'UrlDetalheMateria'],
                                how='left', indicator=True)
df_all[df_all['_merge']=='left_only']

**Conclusão**: Autores possíveis de iniciativas do Senado
- Senador
- Comissão
- CPI

Novo código:
```
# df_pl_2019[~df_pl_2019['Autor'].str.contains("Câmara dos Deputados")]
```

# Testar aplicação em anos anteriores:

<H1> Coleta da Pesquisa Básica das Matérias
<H3> (PLs do Senado de 2018
)

In [ ]:
PARAMS = {
    "sigla":"pls",
    "ano":"2018"
        }

HEADERS = {
    "Accept":"application/json"
}
URL = "https://legis.senado.leg.br/dadosabertos/materia/pesquisa/lista"
response = requests.get(url=URL,params=PARAMS,headers=HEADERS).json()

In [ ]:
#checar se tem projeto de comissão (iniciativa coletiva)
df_pls_2018 = pd.DataFrame(list(response["PesquisaBasicaMateria"]["Materias"]["Materia"]))
df_pls_2018_comissao = df_pls_2018[df_pls_2018['Autor'].str.contains("Comiss")]
df_pls_2018_comissao

In [ ]:
#checar projetos do Senado
df_pls_2018_sen = df_pls_2018[df_pls_2018['Autor'].str.contains("Senador")]
df_pls_2018_sen

In [ ]:
#checar projetos da CD
df_pls_2018_cd = df_pls_2018[df_pls_2018['Autor'].str.contains("Câmara dos Deputados")]
df_pls_2018_cd

,Codigo,IdentificacaoProcesso,DescricaoIdentificacao,Sigla,Numero,Ano,Ementa,Autor,Data,UrlDetalheMateria


In [ ]:
print("projetos de comissao:", len(df_pls_2018_comissao.index))
print("projetos de senadores individuais:", len(df_pls_2018_sen.index))
print("projetos da Câmara dos Dep.:", len(df_pls_2018_cd.index))
print("projetos totais de 2019:", len(df_pls_2018.index))

projetos de comissao: 28
projetos de senadores individuais: 483
projetos da Câmara dos Dep.: 0
projetos totais de 2019: 547


In [ ]:
df_pls_2018.columns

Index(['Codigo', 'IdentificacaoProcesso', 'DescricaoIdentificacao', 'Sigla',
       'Numero', 'Ano', 'Ementa', 'Autor', 'Data', 'UrlDetalheMateria'],
      dtype='object')

In [ ]:
df_pls_2018_sen_comissao = pd.concat([df_pls_2018_sen, df_pls_2018_comissao])
df_all_2018 = df_pls_2018.merge(df_pls_2018_sen_comissao,
                                on=['Codigo', 'IdentificacaoProcesso', 'DescricaoIdentificacao', 'Sigla',
                                    'Numero', 'Ano', 'Ementa', 'Autor', 'Data', 'UrlDetalheMateria'],
                                how='left', indicator=True)
df_all_2018[df_all_2018['_merge']=='left_only']

**Conclusão**: De 2018 para trás não precisa de filtro. A própria sigla "PL**S**" já filtra

<H1> Coletar os detalhes das matérias
<H3> (Temas dos PLs)

In [ ]:
# scraping / coleta dados
PARAMS = {}
HEADERS = {
    "Accept":"application/json"
    }
URL = "https://legis.senado.leg.br/dadosabertos/materia/137600"
response = requests.get(url=URL,params=PARAMS,headers=HEADERS).json()


In [ ]:
response

{'DetalheMateria': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://legis.senado.leg.br/dadosabertos/dados/DetalheMateriav7.xsd',
  'Metadados': {'Versao': '18/06/2023 13:26:14',
   'VersaoServico': '7',
   'DataVersaoServico': '2021-09-14',
   'DescricaoDataSet': 'Detalhes da matéria. ATENÇÃO: para pesquisar requerimentos de comissão por sigla, número e ano, é necessário informar também a sigla da comissão como parâmetro. Ex.: req/1/2021?comissao=cae.'},
  'Materia': {'IdentificacaoMateria': {'CodigoMateria': '137600',
    'SiglaCasaIdentificacaoMateria': 'SF',
    'SiglaSubtipoMateria': 'PL',
    'DescricaoSubtipoMateria': 'Projeto de Lei',
    'NumeroMateria': '03914',
    'AnoMateria': '2019',
    'DescricaoObjetivoProcesso': 'Iniciadora',
    'DescricaoIdentificacaoMateria': 'PL 3914/2019',
    'IndicadorTramitando': 'Sim',
    'IdentificacaoProcesso': '7777031'},
   'DadosBasicosMateria': {'EmentaMateria': 'Altera a Lei n° 7.

In [ ]:
for key in response.keys():
  print(key)

DetalheMateria


In [ ]:
pd.DataFrame(list(response["DetalheMateria"]["Materia"]))

,0
0,IdentificacaoMateria
1,DadosBasicosMateria
2,Classificacoes
3,Assunto
4,OrigemMateria
5,OutrasInformacoes


In [ ]:
list_info = ['IdentificacaoMateria','DadosBasicosMateria','Classificacoes','Assunto','OrigemMateria','OutrasInformacoes']
for info in list_info:
  print(info)
  display(pd.DataFrame(list(response["DetalheMateria"]["Materia"][info])))
  print('\n #------------------#')

IdentificacaoMateria


,0
0,CodigoMateria
1,SiglaCasaIdentificacaoMateria
2,SiglaSubtipoMateria
3,DescricaoSubtipoMateria
4,NumeroMateria
5,AnoMateria
6,DescricaoObjetivoProcesso
7,DescricaoIdentificacaoMateria
8,IndicadorTramitando
9,IdentificacaoProcesso



 #------------------#
DadosBasicosMateria


,0
0,EmentaMateria
1,ExplicacaoEmentaMateria
2,IndexacaoMateria
3,Autor
4,CasaIniciadoraNoLegislativo
5,IndicadorComplementar
6,DataApresentacao
7,DataLeitura
8,NaturezaMateria



 #------------------#
Classificacoes


...


 #------------------#
OrigemMateria


,0
0,NomePoderOrigem
1,SiglaCasaOrigem



 #------------------#
OutrasInformacoes


,0
0,Servico



 #------------------#


In [ ]:
pd.DataFrame(list(response["DetalheMateria"]["Materia"]["Classificacoes"]["Classificacao"]))

,CodigoClasse,DescricaoClasse,DescricaoClasseHierarquica
0,33804850,"Indústria, Comércio e Serviços","Economia e Desenvolvimento / Indústria, Comérc..."


In [ ]:
pd.DataFrame(list(response["DetalheMateria"]["Materia"]["Assunto"]["AssuntoEspecifico"]))

,0
0,Codigo
1,Descricao


In [ ]:
pd.DataFrame(list(response["DetalheMateria"]["Materia"]["Assunto"]["AssuntoGeral"]))

,0
0,Codigo
1,Descricao


In [ ]:
pd.DataFrame(list(response["DetalheMateria"]["Materia"]["OutrasInformacoes"]["Servico"]))

,NomeServico,DescricaoServico,UrlServico
0,AtualizacaoMateria,Mostra as atualizações mais recentes da matéria.,http://legis.senado.leg.br/dadosabertos/materi...
1,AutoriaMateria,Lista o(s) autor(es) da matéria.,http://legis.senado.leg.br/dadosabertos/materi...
2,EmendaMateria,Emendas da matéria,http://legis.senado.leg.br/dadosabertos/materi...
3,MovimentacaoMateria,"Informes legislativos, movimentações, despacho...",http://legis.senado.leg.br/dadosabertos/materi...
4,RelatoriaMateria,Relatoria atual e relatorias encerradas (histó...,http://legis.senado.leg.br/dadosabertos/materi...
5,SituacaoAtualMateria,Traz a situação e a localização atuais de uma ...,http://legis.senado.leg.br/dadosabertos/materi...
6,TextoMateria,Textos da matéria. \nEvoluções da versão: \n28...,http://legis.senado.leg.br/dadosabertos/materi...
7,VotacaoMateria,Votações da matéria. Atenção: o conteúdo da de...,http://legis.senado.leg.br/dadosabertos/materi...
8,VotacoesComissao,NaN,http://legis.senado.leg.br/dadosabertos/votaca...


In [ ]:
pd.DataFrame([response["DetalheMateria"]["Materia"]["DadosBasicosMateria"]])

,EmentaMateria,ExplicacaoEmentaMateria,IndexacaoMateria,Autor,CasaIniciadoraNoLegislativo,IndicadorComplementar,DataApresentacao,DataLeitura,NaturezaMateria
0,Estabelece normas e diretrizes relativas à cad...,Define animal de estimação como aquele criado ...,"CRIAÇÃO , LEI FEDERAL , NORMAS , DIRETRIZ ...",Senador Luis Carlos Heinze (PP/RS),SF,Não,2019-12-18,2019-12-18,"{'CodigoNatureza': '11', 'NomeNatureza': 'NORM..."


In [ ]:
pd.DataFrame([response["DetalheMateria"]["Materia"]["DadosBasicosMateria"]["NaturezaMateria"]])

,CodigoNatureza,NomeNatureza,DescricaoNatureza
0,11,NORMA_GERAL,Norma Geral


In [ ]:
pd.DataFrame([response["DetalheMateria"]["Materia"]["Assunto"]["AssuntoEspecifico"]])

,Codigo,Descricao
0,109,"Indústria, comércio e serviço"


In [ ]:
pd.DataFrame([response["DetalheMateria"]["Materia"]["Assunto"]["AssuntoGeral"]])

,Codigo,Descricao
0,82,Econômico


In [ ]:
pd.DataFrame([response["DetalheMateria"]["Materia"]["OrigemMateria"]])

,NomePoderOrigem,SiglaCasaOrigem
0,Legislativo,SF


In [ ]:
response

{'DetalheMateria': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://legis.senado.leg.br/dadosabertos/dados/DetalheMateriav7.xsd',
  'Metadados': {'Versao': '24/05/2023 10:22:40',
   'VersaoServico': '7',
   'DataVersaoServico': '2021-09-14',
   'DescricaoDataSet': 'Detalhes da matéria. ATENÇÃO: para pesquisar requerimentos de comissão por sigla, número e ano, é necessário informar também a sigla da comissão como parâmetro. Ex.: req/1/2021?comissao=cae.'},
  'Materia': {'IdentificacaoMateria': {'CodigoMateria': '140348',
    'SiglaCasaIdentificacaoMateria': 'SF',
    'SiglaSubtipoMateria': 'PL',
    'DescricaoSubtipoMateria': 'Projeto de Lei',
    'NumeroMateria': '06590',
    'AnoMateria': '2019',
    'DescricaoObjetivoProcesso': 'Iniciadora',
    'DescricaoIdentificacaoMateria': 'PL 6590/2019',
    'IndicadorTramitando': 'Sim',
    'IdentificacaoProcesso': '7857980'},
   'DadosBasicosMateria': {'EmentaMateria': 'Estabelece normas 

In [ ]:
### coleta e processamento por materia (codigo)
df_final = pd.DataFrame()
total_codigos = len(codigo_list)
count = 0

for c in codigo_list:
  # acompanhamento loop
  count = count + 1
  print("Código:", c, "-", count,"/",total_codigos)

  # scraping / coleta dados
  PARAMS = {}
  HEADERS = {
      "Accept":"application/json"
      }
  URL = "https://legis.senado.leg.br/dadosabertos/materia/" + str(c)
  response = requests.get(url=URL,params=PARAMS,headers=HEADERS).json()

  # processamento dados
  df_identmat = pd.DataFrame([response["DetalheMateria"]["Materia"]["IdentificacaoMateria"]])

  df_basic = pd.DataFrame([response["DetalheMateria"]["Materia"]["DadosBasicosMateria"]])
  df_basic = df_basic.drop('NaturezaMateria', axis=1)
  df_basic['CodigoMateria'] = str(c)
  df_basic_nat = pd.DataFrame([response["DetalheMateria"]["Materia"]["DadosBasicosMateria"]["NaturezaMateria"]])
  df_basic_nat['CodigoMateria'] = str(c)

  df_clas = pd.DataFrame(response["DetalheMateria"]["Materia"]["Classificacoes"]["Classificacao"])
  df_clas['CodigoMateria'] = str(c)

  df_ass_esp = pd.DataFrame([response["DetalheMateria"]["Materia"]["Assunto"]['AssuntoEspecifico']])
  df_ass_esp = df_ass_esp.rename(columns={"Codigo": "Codigo_especifico", "Descricao": "Descricao_especifica"})
  df_ass_esp['CodigoMateria'] = str(c)

  df_ass_geral = pd.DataFrame([response["DetalheMateria"]["Materia"]["Assunto"]['AssuntoGeral']])
  df_ass_geral = df_ass_geral.rename(columns={"Codigo": "Codigo_geral", "Descricao": "Descricao_geral"})
  df_ass_geral['CodigoMateria'] = str(c)

  df_join1 = pd.merge(df_identmat, df_basic, how="left", on=["CodigoMateria"])
  df_join2 = pd.merge(df_join1, df_basic_nat, how="left", on=["CodigoMateria"])
  df_join3 = pd.merge(df_join2, df_clas, how="left", on=["CodigoMateria"])
  df_join4 = pd.merge(df_join3, df_ass_esp, how="left", on=["CodigoMateria"])
  df_result = pd.merge(df_join4, df_ass_geral, how="left", on=["CodigoMateria"])

  df_final = pd.concat([df_final, df_result])

Código: 137906 - 1 / 1149
Código: 140149 - 2 / 1149
Código: 140146 - 3 / 1149
Código: 138227 - 4 / 1149
Código: 136745 - 5 / 1149
Código: 136659 - 6 / 1149
Código: 136631 - 7 / 1149
Código: 135940 - 8 / 1149
Código: 135791 - 9 / 1149
Código: 135779 - 10 / 1149
Código: 135564 - 11 / 1149
Código: 135460 - 12 / 1149
Código: 135171 - 13 / 1149
Código: 138839 - 14 / 1149
Código: 138312 - 15 / 1149
Código: 139968 - 16 / 1149
Código: 139967 - 17 / 1149
Código: 139268 - 18 / 1149
Código: 138997 - 19 / 1149
Código: 138993 - 20 / 1149
Código: 137121 - 21 / 1149
Código: 136980 - 22 / 1149
Código: 136978 - 23 / 1149
Código: 136859 - 24 / 1149
Código: 136641 - 25 / 1149
Código: 138076 - 26 / 1149
Código: 137062 - 27 / 1149
Código: 136481 - 28 / 1149
Código: 135969 - 29 / 1149
Código: 135810 - 30 / 1149
Código: 135803 - 31 / 1149
Código: 135802 - 32 / 1149
Código: 135798 - 33 / 1149
Código: 135487 - 34 / 1149
Código: 135422 - 35 / 1149
Código: 135233 - 36 / 1149
Código: 135232 - 37 / 1149
Código: 13

In [ ]:
df_final.to_csv('pl_senado_2019.csv')

In [ ]:
df_final.columns

Index(['CodigoMateria', 'SiglaCasaIdentificacaoMateria', 'SiglaSubtipoMateria',
       'DescricaoSubtipoMateria', 'NumeroMateria', 'AnoMateria',
       'DescricaoObjetivoProcesso', 'DescricaoIdentificacaoMateria',
       'IndicadorTramitando', 'IdentificacaoProcesso', 'EmentaMateria',
       'ExplicacaoEmentaMateria', 'IndexacaoMateria', 'Autor',
       'CasaIniciadoraNoLegislativo', 'IndicadorComplementar',
       'DataApresentacao', 'DataLeitura', 'CodigoNatureza', 'NomeNatureza',
       'DescricaoNatureza', 'CodigoClasse', 'DescricaoClasse',
       'DescricaoClasseHierarquica', 'Codigo_especifico',
       'Descricao_especifica', 'Codigo_geral', 'Descricao_geral',
       'CodMateriaSucessoraNoSenado', 'ApelidoMateria'],
      dtype='object')

In [ ]:
df_final.head()

,CodigoMateria,SiglaCasaIdentificacaoMateria,SiglaSubtipoMateria,DescricaoSubtipoMateria,NumeroMateria,AnoMateria,DescricaoObjetivoProcesso,DescricaoIdentificacaoMateria,IndicadorTramitando,IdentificacaoProcesso,...,DescricaoNatureza,CodigoClasse,DescricaoClasse,DescricaoClasseHierarquica,Codigo_especifico,Descricao_especifica,Codigo_geral,Descricao_geral,CodMateriaSucessoraNoSenado,ApelidoMateria
0,137906,SF,PL,Projeto de Lei,04242,2019,Iniciadora,PL 4242/2019,Não,7783022,...,Norma Geral,33804940,Tributos,Economia e Desenvolvimento / Tributos,117,Tributação,82,Econômico,NaN,NaN
0,140149,SF,PL,Projeto de Lei,06401,2019,Iniciadora,PL 6401/2019,Não,7852195,...,Norma Geral,33805617,Direito Penal e Penitenciário,Jurídico / Direito Penal e Penitenciário,130,Direito penal e processual penal,83,Jurídico,NaN,NaN
0,140146,SF,PL,Projeto de Lei,06400,2019,Iniciadora,PL 6400/2019,Não,7852165,...,Norma Geral,33805617,Direito Penal e Penitenciário,Jurídico / Direito Penal e Penitenciário,130,Direito penal e processual penal,83,Jurídico,NaN,NaN
0,138227,SF,PL,Projeto de Lei,04640,2019,Iniciadora,PL 4640/2019,Sim,7791666,...,Norma Geral,33805617,Direito Penal e Penitenciário,Jurídico / Direito Penal e Penitenciário,130,Direito penal e processual penal,83,Jurídico,NaN,NaN
0,136745,SF,PL,Projeto de Lei,02879,2019,Iniciadora,PL 2879/2019,Sim,7752219,...,Norma Geral,33805422,Processo Penal,Jurídico / Processo / Processo Penal,130,Direito penal e processual penal,83,Jurídico,NaN,NaN
